In [1]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
import numpy as np
import torch
import pandas as pd
import time
import os
import json
from scipy.special import softmax

[nltk_data] Downloading package punkt to
[nltk_data]     /home/p.bushipaka/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
train = pd.read_csv('../../../data/chunked_train.csv', index_col=0)
val = pd.read_csv('../../../data/chunked_val.csv', index_col=0)
test = pd.read_csv('../../../data/chunked_test.csv', index_col=0)

In [6]:
print(train.shape)
print(val.shape)
print(test.shape)

(2965, 22)
(608, 22)
(618, 22)


In [7]:
train['label'] = train[['answer1', 'answer2', 'answer3']].apply(lambda x: x.dropna().tolist(), axis=1)
train = train.explode('label').reset_index(drop=True)

val['label'] = val[['answer1', 'answer2', 'answer3']].apply(lambda x: x.dropna().tolist(), axis=1)
val = val.explode('label').reset_index(drop=True)

In [8]:
train = train.loc[train['label'] != 'Link-broken']
val = val.loc[val['label'] != 'Link-broken']
#test = test.loc[test['label'] != 'Link-broken']

In [9]:
labels = train['label'].unique().tolist()
labels = [s.strip() for s in labels ]


NUM_LABELS= len(labels)

id2label={id:label for id,label in enumerate(labels)}

label2id={label:id for id,label in enumerate(labels)}

In [10]:
print(id2label)
print(label2id)

{0: 'Pro', 1: 'Neutral', 2: 'Not-about', 3: 'Against'}
{'Pro': 0, 'Neutral': 1, 'Not-about': 2, 'Against': 3}


In [11]:
train["labels"] = train['label'].map(lambda x: label2id[x.strip()])
val["labels"] = val['label'].map(lambda x: label2id[x.strip()])

In [12]:
train_df = train[['chunks', 'labels']] 
eval_df = val[['chunks', 'labels']]


In [13]:
save_dir = '../output/'
model_name = 'google-bert/bert-base-uncased'
model_name_filename = model_name.replace("/", "-")

In [37]:
start_time = time.time()
logging.basicConfig(level=logging.DEBUG)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.INFO)

from simpletransformers.classification import ClassificationModel, ClassificationArgs

# giving max_input_length (based on model) and batch size
max_seq_length = 512
batch_size = 32 # change it if there is Out of Memory (OOM) error



# setting optional model configuration
model_args = ClassificationArgs()
model_args.num_train_epochs = 4



model_args.overwrite_output_dir = True
model_args.best_model_dir = f'./output/mp_chunk_{model_name_filename}/best_model'
model_args.reprocess_input_data = True


model_args.evaluate_during_training = True
model_args.early_stopping_metric = 'eval_loss'  # eval_loss The metric that should be used with early stopping. (Should be computed during eval_during_training)
model_args.use_early_stopping = True
model_args.max_seq_length = max_seq_length
model_args.manual_seed = 42 # for reproducible results
model_args.do_lower_case = True # when we are using BERT Base/large uncased. converts to lower case
model_args.train_batch_size = batch_size
model_args.eval_batch_size = batch_size

model_args.save_eval_checkpoints =  False
model_args.save_model_every_epoch = False
#model_args.sliding_window = True #for chunking
#model_args.stride = 100  # this is how much overlap we need between two chunks
#model_args.tie = 1
model_args.save_to_hub = True

# Create a ClassificationModel
model = ClassificationModel(
    "bert",
    "google-bert/bert-base-uncased",
    num_labels=4,
    args=model_args,
    use_cuda=True,
    cuda_device=3,
    
)

# Train model
_, training_details = model.train_model(train_df, eval_df = eval_df)


# show training details
logging.info('training details')
logging.info(pd.DataFrame(training_details))

model = ClassificationModel(model.args.model_type, model.args.best_model_dir)

end_time = time.time()
total_time = end_time - start_time
total_time_minutes, total_time_seconds = divmod(total_time, 60)
print(f'total time taken for the finetuning of {model_name}: {total_time}')
time_data = {
        "total_time": {
            "minutes": int(total_time_minutes),
            "seconds": round(total_time_seconds, 2)
        }
}
t_filename = f"time_{model_name_filename}.json"
t_filepath = os.path.join(save_dir, t_filename)
with open(t_filepath, 'w') as json_file:
    json.dump(time_data, json_file)

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /google-bert/bert-base-uncased/resolve/main/config.json HTTP/1.1" 200 0
loading configuration file config.json from cache at /home/p.bushipaka/.cache/huggingface/hub/models--google-bert--bert-base-uncased/snapshots/86b5e0934494bd15c9632b12f734a8a67f723594/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "p

  0%|          | 0/8840 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_bert_512_4_2


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/277 [00:00<?, ?it/s]

...

Running Epoch 3 of 4:   0%|          | 0/277 [00:00<?, ?it/s]

/home/p.bushipaka/miniconda3/envs/hlt/lib/python3.11/site-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1820 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_512_4_2
Configuration saved in outputs/config.json
Model weights saved in outputs/model.safetensors
tokenizer config file saved in outputs/tokenizer_config.json
Special tokens file saved in outputs/special_tokens_map.json
INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to outputs/.
INFO:root:training details
INFO:root:   global_step  train_loss       mcc  eval_loss
0          277    1.008148  0.148067   1.347283
1          554    0.845306  0.185806   1.323618
2          831    1.050674  0.188495   1.417460
3         1108    0.716927  0.189199   1.513732
loading configuration file ./output/mp_chunk_google-bert-bert-base-uncased/best_model/config.json
Model config BertConfig {
  "_name_or_path": "google-bert/bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_pro

total time taken for the finetuning of google-bert/bert-base-uncased: 757.0849359035492


In [38]:
from sklearn.metrics import f1_score, accuracy_score

def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')

result, model_outputs, wrong_predictions = model.eval_model(eval_df, f1=f1_multiclass, acc=accuracy_score)

/home/p.bushipaka/miniconda3/envs/hlt/lib/python3.11/site-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1820 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_512_4_2


Running Evaluation:   0%|          | 0/57 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.1858064060620813, 'f1': 0.3956043956043956, 'acc': 0.3956043956043956, 'eval_loss': 1.3236177678693806}


### prediction on test set

In [14]:
test = pd.read_csv('../../data/chunked_test.csv', index_col=0)
test.shape

(618, 22)

In [16]:
labels = train['label'].unique().tolist()
labels = [s.strip() for s in labels ]


NUM_LABELS= len(labels)

id2label={id:label for id,label in enumerate(labels)}

label2id={label:id for id,label in enumerate(labels)}

In [17]:
save_dir = '../output/'
model_name = 'google-bert/bert-base-uncased'
model_name_filename = model_name.replace("/", "-")

In [18]:
model = ClassificationModel(
    "bert", f'./output/mp_chunk_{model_name_filename}/best_model', use_cuda = True
)

In [19]:
test.shape

(618, 22)

In [20]:
def make_predictions(row):
    doc = row['chunks']
    predictions, raw_outputs = model.predict(doc)
    prediction_id = predictions[0]
    probabilities = softmax(raw_outputs, axis=1)
    # Map prediction ID to label
    predicted_label = id2label[prediction_id]
    # Create dictionary of labels and probabilities
    prediction_dict = {label: prob for label, prob in zip(id2label.values(), probabilities[0])}
    # Return both prediction and probabilities
    return predicted_label, probabilities[0], prediction_dict

predictions = test.apply(make_predictions, axis=1, result_type='expand')
test = test.assign(
    predicted_label=predictions[0],
    probabilities=predictions[1],
    prediction_dict=predictions[2]
)

  0%|          | 0/2317 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2312 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1354 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2233 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/514 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
mode_predictions = test.groupby('docID')['predicted_label'].apply(lambda x: x.mode()[0]).reset_index()

# Calculate sum of predicted probabilities for each class for each docID
sum_probabilities = test.groupby('docID')['probabilities'].apply(lambda x: np.sum(x)).reset_index()

# Merge mode predictions and sum probabilities back to the test DataFrame
test = pd.merge(test, mode_predictions, on='docID', suffixes=('', '_mode'))
test = pd.merge(test, sum_probabilities, on='docID', suffixes=('', '_sum'))

In [22]:
test.to_csv('../results/mp_chunk_test_results.csv', index = False)

In [2]:
#test = pd.read_csv('../results/mp_chunk_test_results.csv')

In [3]:
test.drop_duplicates('docID', inplace = True)
test = test.loc[test['majority_label'] != 'No majority']

In [6]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

y_true = test['majority_label']
y_pred = test['predicted_label_mode']

# Accuracy
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy*100)

# Precision
precision = precision_score(y_true, y_pred, average='macro')  # 'macro' averaging for multiclass
print("Precision:", precision*100)

# Recall
recall = recall_score(y_true, y_pred, average='macro')  # 'macro' averaging for multiclass
print("Recall:", recall*100)

# F1 Score
f1 = f1_score(y_true, y_pred, average='macro')  # 'macro' averaging for multiclass
print("F1 Score:", f1*100)

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Classification Report
class_report = classification_report(y_true, y_pred)
print("Classification Report:")
print(class_report)


Accuracy: 47.482014388489205
Precision: 53.91706412294648
Recall: 49.8680165731088
F1 Score: 50.2126754303211
Confusion Matrix:
[[18  8  1  2]
 [11 22  0 10]
 [ 3  6 14  1]
 [ 5 24  2 12]]
Classification Report:
              precision    recall  f1-score   support

     Against       0.49      0.62      0.55        29
     Neutral       0.37      0.51      0.43        43
   Not-about       0.82      0.58      0.68        24
         Pro       0.48      0.28      0.35        43

    accuracy                           0.47       139
   macro avg       0.54      0.50      0.50       139
weighted avg       0.51      0.47      0.47       139

